In [3]:
import queueing_tool as qt
import networkx as nx
import numpy 
import matplotlib
import csv, json
#import pygraphviz

In [13]:

class System:
 
    def __init__(self, stop_id, stop_code, stop_name, stop_desc, platform_code, platform_name,\
                    stop_lat, stop_lon, stop_address, zone_id, stop_url, level_id, location_type,\
                    parent_station, wheelchair_boarding):
        
        self.stop_id = stop_id
        self.stop_code = stop_code
        self.stop_name = stop_name
        self.stop_desc = stop_desc
        self.platform_code = platform_code
        self.platform_name = platform_name
        self.stop_lat = float(stop_lat)
        self.stop_lon = float(stop_lon)
        self.stop_address = stop_address
        self.zone_id = zone_id
        self.stop_url = stop_url
        self.level_id = level_id
        self.location_type = location_type
        self.parent_station = parent_station
        self.wheelchair_boarding = wheelchair_boarding

        
        if parent_station == '':
            self.identifier = stop_id
        else:
            self.identifier = parent_station
        
    def __str__(self):
        if self.parent_station == '':
            string = 'Name: ' + self.stop_name + '\nStop ID: ' + self.stop_id + \
                '\nCoordinates: ' + '(' + self.stop_lat + ', ' + self.stop_lon + ')'
        else:
            string = 'Name: ' + self.stop_name + '\nParent Station: ' + \
                self.parent_station + '\nCoordinates: ' + '(' + str(self.stop_lat) \
                + ', ' + str(self.stop_lon) + ')'
        return string
    
    def get_name(self):
        return self.stop_name
    
    def get_id(self):
        return self.identifier
    
    def get_parent_station(self):
        return self.parent_station
    
    def get_coordinates(self):
        return (self.stop_lon, self.stop_lat)
    
   
            

def create_system(filename='stops.csv'):
 
    with open(filename, newline='') as stops:
        station_info = csv.reader(stops) 
       
        next(station_info)
       
        stations = []
        
        for row in station_info:
            (stop_id, stop_code, stop_name, stop_desc, platform_code, platform_name,\
                stop_lat, stop_lon, stop_address, zone_id, stop_url, level_id,\
                location_type, parent_station, wheelchair_boarding) = tuple(row)
            stations.append(Station(stop_id, stop_code, stop_name, stop_desc, \
                platform_code, platform_name, stop_lat, stop_lon, stop_address, \
                zone_id, stop_url, level_id, location_type, parent_station,\
                wheelchair_boarding))
        
    return stations

In [12]:
import networkx as nx

def build_system_network(filename = "stops.csv"):
  
    # get lon, lat locations
    locations = {} # location dictionary
    for station in create_system():
        locations[station.get_id()] = station.get_coordinates()
        
    # initialize networkx graph
    bus_map = nx.Graph()
    
    # still need to create a file for edges between nodes
    with open(filename) as t:
         # read in all lines 
        edges = t.readlines()
         # iterate over edges and add all nodes and edges to graph
        for line in edges:
             (source, destination, time, color) = tuple(line.split(','))
             # add edge to graph with color attribute and time as weight
             bus_map.add_edge(source, destination, weight=float(time), color=color) 
    
    # return graph
    return bus_map

def draw_network(G, edges, color='black', new_plot=True, title='Map of T', label=None):
    '''
    Creates plot of network with stop positions
    '''
    # get lon, lat locations
    locations = {} # location dictionary
    nodes = list(chain(*edges)) # get list of nodes
    for station in create_system():
        locations[station.get_id()] = station.get_coordinates()
    
    # plot
    if new_plot:
        plt.figure()
        plt.title(title)    
    
    if label is not None:
        nx.draw_networkx_labels(G, pos=locations, labels=label, font_size=6)
    # draw edges with geo positions 
    nx.draw_networkx_edges(G, pos=locations, edge_color=color, edgelist=edges)
    # draw nodes based on geo positions
    nx.draw_networkx_nodes(G, pos=locations, node_color='black',\
                           node_size=20, nodelist=nodes)

# graph of all stops and edges between them                 
G_map = build_system_network()
# draw map with colors corresponding to routes
edges_map,colors = zip(*nx.get_edge_attributes(G_map,'color').items())
draw_network(G_map,edges_map,color=colors)

ValueError: too many values to unpack (expected 15)

In [11]:
def avg_shortest_path(demand, network_with_xfer):
    '''
    calculates average shortest path on the system
    '''
    
    edges_demand,demands = zip(*nx.get_edge_attributes(demand,'weight').items())
    demand_dict = dict(zip(edges_demand, demands))
    shortest_path_length = nx.shortest_path_length(network_with_xfer,weight='weight')
    
    sp_pairs = list()
    sp_length = list()
    for item in shortest_path_length: # item is tuple 
        start_node = item[0] # get start node
        start_node_data = item[1] 
        for dest_node,len_sp in start_node_data.items():
                sp_pairs.append((start_node,dest_node))
                sp_length.append(len_sp)

    # data cleaning
    for i in range(len(sp_pairs)):
        sp_pair = sp_pairs[i]
        o = sp_pair[0] # origin 
        d = sp_pair[1] #dest
        if '_' in o:
            o = o[:o.find('_')] 
        if '_' in d:
            d = d[:d.find('_')]
        sp_pair = (o,d)
        sp_pairs[i] = sp_pair 
        
    sorted_lengths = sorted(sp_length, reverse=True)
    sorted_pairs = [pair for _, pair in sorted(zip(sp_length,sp_pairs),\
                                               key=lambda pair: pair[0], reverse=True)]

 
    sp_dict = dict(zip(sorted_pairs, sorted_lengths))
    
    # weighted average
    total_weight = sum(demands) 
    weighted_travel_times = {} 
    for key in demand_dict:
        if key in sp_dict: 
          
            weighted_travel_times[key] = demand_dict[key]*sp_dict[key] 
        
    avg_shortest_path = sum(weighted_travel_times.values())/total_weight
    
 
    return avg_shortest_path  

In [10]:

def distance(locations,o,d):

    olat,olon = locations[o]
    dlat,dlon = locations[d]
    distance = (np.hypot(abs(olat-dlat),abs(olon-dlon)))
    return distance

